<a href="https://colab.research.google.com/github/prakhar-chaurasiya/final_project_nism/blob/main/trade_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

to do


*   
*   



In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import tensorflow as tf
np.set_printoptions(suppress=True)

dlf_file = '/content/drive/MyDrive/Colab Notebooks/FA/HIGHFREQ-DATA/trade_data/DLF.trade'
trade_df = pd.read_csv(dlf_file, sep='|', header=None)
trade_df.columns =['record', 'segment', 'trade_no', 
                    'trade_time', 'series', 'trade_price', 
                    'trade_qty', 'buy_order_no', 'buy_algo', 
                    'client', 'sell_order_no', 'sell_algo']
jiffy = 65536
ist = 330*60
trade_df['time'] = trade_df['trade_time'].apply(lambda x: datetime.fromtimestamp(x/jiffy))#.strftime('%Y-%m-%d %H:%M:%S'))
# trade_df['min'] = trade_df['trade_time'].apply(lambda x: datetime.fromtimestamp(x/jiffy-ist).strftime('%H:%M:%S'))
trade_df['sec'] = pd.to_datetime(trade_df['time']).dt.strftime('%H%M%S').astype('int')
trade_df['price_*_qty'] = trade_df['trade_qty'] * trade_df['trade_price']
trade_df

,record,segment,trade_no,trade_time,series,trade_price,trade_qty,buy_order_no,buy_algo,client,sell_order_no,sell_algo,time,sec,price_*_qty
0,PO,CASH,2021012000009187,84908499166667,EQ,29105,50,1000000000110793,1,3,1000000000142464,1,2011-01-21 09:07:58.397629,90758,1455250
1,PO,CASH,2021012000009188,84908499166668,EQ,29105,100,1000000000107670,1,3,1000000000142464,1,2011-01-21 09:07:58.397644,90758,2910500
2,PO,CASH,2021012000009189,84908499166669,EQ,29105,9,1000000000090727,1,3,1000000000142464,1,2011-01-21 09:07:58.397659,90758,261945
3,PO,CASH,2021012000009190,84908499166670,EQ,29105,9,1000000000090730,1,3,1000000000142464,1,2011-01-21 09:07:58.397675,90758,261945
4,PO,CASH,2021012000009191,84908499166671,EQ,29105,140,1000000000067699,1,3,1000000000142464,1,2011-01-21 09:07:58.397690,90758,4074700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100817,RM,CASH,2021012004504238,84910090545595,EQ,29045,200,1000000016682060,1,3,1000000016678737,1,2011-01-21 15:52:40.913010,155240,5809000
100818,RM,CASH,2021012004504262,84910092577977,EQ,29045,200,1000000016682123,1,3,1000000016678737,1,2011-01-21 15:53:11.924698,155311,5809000
100819,RM,CASH,2021012004504279,84910092944059,EQ,29045,7,1000000016682135,1,2,1000000016678737,1,2011-01-21 15:53:17.510666,155317,203315
100820,RM,CASH,2021012004504453,84910101941838,EQ,29045,34,1000000016682442,1,3,1000000016678737,1,2011-01-21 15:55:34.805878,155534,987530


weighted trade price, volume, total trades of a second

In [28]:
trade_np = trade_df[['sec', 'price_*_qty', 'trade_qty']].values
rows, row_idx = np.unique(trade_np[:, 0], return_index=True)
wpvtt = np.zeros((rows.shape[0], 4), dtype=np.float32)
for i, (r, id1, id2) in enumerate(zip(rows, row_idx, row_idx[1:])):
    wpvtt[i, 0] = r
    wpvtt[i, 1] = trade_np[id1:id2, 1:2].T.sum()/trade_np[id1:id2, 2:].T.sum()
    wpvtt[i, 2] = trade_np[id1:id2, 2:].T.sum()
    wpvtt[i, 3] = trade_np[id1:id2, 2:].shape[0]


temp = trade_df.groupby('sec').get_group(trade_df['sec'].unique()[-1])[['trade_qty', 'price_*_qty']].sum()
wpvtt[-1, 0] = trade_df['sec'].unique()[-1]
wpvtt[-1, 1] = temp[1]/temp[0]
wpvtt[-1, 2] = temp[0]
wpvtt[-1, 3] = trade_df.groupby('sec').get_group(trade_df['sec'].unique()[-1])[['trade_qty', 'price_*_qty']].shape[0]
wpvtt

array([[ 90758.   ,  29105.   ,  20847.   ,    274.   ],
       [ 91500.   ,  29108.145,   4905.   ,     46.   ],
       [ 91501.   ,  29116.441,   5626.   ,     65.   ],
       ...,
       [155317.   ,  29045.   ,      7.   ,      1.   ],
       [155534.   ,  29045.   ,     34.   ,      1.   ],
       [155708.   ,  29045.   ,     10.   ,      1.   ]], dtype=float32)

ohlc of trade

In [11]:
prices_np = trade_df[['sec', 'trade_price']].values
prices_tf = tf.convert_to_tensor(prices_np, dtype=tf.float32)
rows, row_idx = np.unique(prices_np[:, 0], return_inverse=True)
ohlc = np.zeros((rows.shape[0], 5), dtype=np.float32)
for i, r in enumerate(rows):
    p = prices_tf[prices_tf[:, 0]==r][:, 1].numpy()
    o = p[0]
    h = p.max()
    l = p.min()
    c = p[-1]
    ohlc[i, 0] = r
    ohlc[i, 1] = o
    ohlc[i, 2] = h
    ohlc[i, 3] = l
    ohlc[i, 4] = c
    print(r)
ohlc

array([[ 90758.,  29105.,  29105.,  29105.,  29105.],
       [ 91500.,  29100.,  29130.,  29100.,  29100.],
       [ 91501.,  29130.,  29165.,  29095.,  29110.],
       ...,
       [155317.,  29045.,  29045.,  29045.,  29045.],
       [155534.,  29045.,  29045.,  29045.,  29045.],
       [155708.,  29045.,  29045.,  29045.,  29045.]], dtype=float32)

volume